# 上下文增强检索
检索增强生成（RAG）通过从外部来源检索相关知识来增强 AI 的响应。传统的检索方法返回孤立的文本片段，这可能导致答案不完整。 为了解决这个问题，我们引入了上下文增强检索，确保检索到的信息包括相邻的片段，以实现更好的连贯性。
-----
- 数据采集：从 PDF 中提取文本
- 重叠上下文分块：将文本分割成重叠的块以保留上下文
- 嵌入创建：将文本块转换为数值表示
- 上下文感知检索：检索相关块及其邻居以获得更好的完整性
- 回答生成：使用语言模型根据检索到的上下文生成回答。
- 评估：使用评估数据集评估模型性能。

In [ ]:
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    检索最相关的文本块及其相邻的上下文块

    Args:
    query (str): 搜索查询
    text_chunks (List[str]): 文本块列表
    embeddings (List[dict]): 文本块嵌入列表
    k (int): 要检索的相关块数量
    context_size (int): 包含的相邻块数量

    Returns:
    List[str]: 包含上下文信息的相关文本块
    """
     # 将查询转换为嵌入向量
    query_embedding = create_embeddings(query)
    similarity_scores = []
    # 计算查询与每个文本块嵌入之间的相似度分数
    for i, chunk_embedding in enumerate(embeddings):
        # 计算查询嵌入与当前文本块嵌入之间的余弦相似度
        similarity_score = cosine_similarity(np.array(query_embedding), chunk_embedding)
        # 将索引和相似度分数存储为元组
        similarity_scores.append((i, similarity_score))
        
    # 按相似度分数降序排序（相似度最高排在前面）
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # 获取最相关块的索引
    # top_index = [index for index, _ in similarity_scores[:k]]
    top_index = similarity_scores[0][0]

    # similarities = [cosine_similarity(query_embedding, emb) for emb in embeddings]
    # top_indices = np.argsort(similarities)[-k:][::-1]

    # 定义上下文包含的范围
    # 确保不会超出 text_chunks 的边界
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)

    # 返回最相关的块及其相邻的上下文块
    return [text_chunks[i] for i in range(start, end)]
        